In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sandeepk0mmineni_team_winners_dataset_path = kagglehub.dataset_download('sandeepk0mmineni/team-winners-dataset')

print('Data source import complete.')


In [ ]:
import pandas as pd

df_train_action_history = pd.read_csv('/kaggle/input/team-winners-dataset/train_action_history.csv')
df_train_cdna_data = pd.read_csv('/kaggle/input/team-winners-dataset/train_cdna_data.csv')

 ## Communication History (6 months for 2 lakh customers) :
 ### df_train_action_history
It contains the interaction data of the customer and the offer. Below are the relevant columns in action history:

- `customer_code` : It’s the unique identifier for the customer
- `Offer_id`: Identifier for the offer
- `Offer_subid` : Identifier for the subid of an offer.(One offer_id can have multiple offer_subid)
- `product_category` and `product_sub_category`: Each offer sent is for specific product category and subcategory
- `batch_id`: Identifier for the batch. Helpful in tracking events on same communication.
- `send_timestamp`: Date & time at which the email was sent
- `open_timestamp`: Date & time at which the email was opened. Null implies the email was not opened

In [ ]:
df_train_action_history.head()

In [ ]:
df_train_action_history.shape ## 87 Lakh, 97,911 Rows AND 8 Columns

In [ ]:
df_train_action_history.describe()

In [ ]:
df_train_cdna_data.shape ## 12 Lakh 85,402 Rows AND 303 Columns

In [ ]:
df_train_cdna_data.head()

In [ ]:
# Count the number of null values per row
df_train_cdna_data['null_count'] = df_train_cdna_data.isnull().sum(axis=1)

# Sort by CUSTOMER_CODE and then by null_count (ascending) to prioritize rows with the least null values
df_train_cdna_data_sorted = df_train_cdna_data.sort_values(by=['CUSTOMER_CODE', 'null_count'])

# Drop duplicates, keeping the first (least null) row for each CUSTOMER_CODE
df_train_cdna_data_unique = df_train_cdna_data_sorted.drop_duplicates(subset='CUSTOMER_CODE', keep='first')

# Drop the 'null_count' column as it's no longer needed
df_train_cdna_data_unique = df_train_cdna_data_unique.drop(columns=['null_count'])

# Reset the index
df_train_cdna_data_unique.reset_index(drop=True, inplace=True)

# Display the shape of the resulting DataFrame and a sample
print(df_train_cdna_data_unique.shape)
print(df_train_cdna_data_unique.head())

In [ ]:
df_train_cdna_data = df_train_cdna_data_unique

## Finding Missing Values Distribution

In [ ]:
## Data Distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the number of missing values in each column
missing_values = df_train_cdna_data.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (missing_values / len(df_train_cdna_data)) * 100

# Create a DataFrame to store the results
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

# Sort the columns by the percentage of missing values
missing_data = missing_data.sort_values(by='Percentage', ascending=False)

# Display the first few rows of the missing data
print(missing_data.head())

# Plotting the missing data as a bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_data.index, y='Percentage', data=missing_data)
plt.xticks(rotation=90)
plt.title('Percentage of Missing Values in Each Column')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.show()

## Dropping Columns with Missing Value % above 40

In [ ]:
# Calculate the percentage of missing values for each column
missing_percentage = df_train_cdna_data.isnull().mean() * 100

# Identify columns with 27% or more missing values
columns_to_drop = missing_percentage[missing_percentage >= 40].index

# Drop the identified columns
df_train_cdna_data_cleaned = df_train_cdna_data.drop(columns=columns_to_drop)

# Display the cleaned dataset
print(f"Original shape: {df_train_cdna_data.shape}")
print(f"Cleaned shape: {df_train_cdna_data_cleaned.shape}")
print(f"Columns removed: {list(columns_to_drop)}")

In [ ]:
df_train_cdna_data_cleaned.shape

### Variance Thresholding

In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

# Select only numerical columns
numerical_columns = df_train_cdna_data_cleaned.select_dtypes(include=['number'])
print("Numerical Columns shape: ", numerical_columns.shape)

# Set a threshold (e.g., remove features with variance < 0.01)
selector = VarianceThreshold(threshold=0.01)
X_high_variance = selector.fit_transform(numerical_columns)

# Get retained columns
retained_columns = numerical_columns.columns[selector.get_support()]
print("High-Variance Features:\n", retained_columns)
print("High Variance Features count: ", len(retained_columns))


# Replace numerical columns in the original DataFrame with high-variance features
df_train_cdna_data_cleaned = pd.concat(
    [df_train_cdna_data_cleaned.drop(columns=numerical_columns.columns),
     pd.DataFrame(X_high_variance, columns=retained_columns, index=df_train_cdna_data_cleaned.index)],
    axis=1
)
print("Final CDNA Data Shape: ", df_train_cdna_data_cleaned.shape)

# Get all numerical column names
original_numerical_columns = numerical_columns.columns

# Find dropped columns by taking the difference
dropped_columns = original_numerical_columns.difference(retained_columns)
print("Dropped Columns:\n", dropped_columns)
print("Dropped Columns count: ", len(dropped_columns))

In [ ]:
df_train_cdna_data_cleaned.shape

## Finding Missing Values Distribution on CDNA Cleaned Dataset

In [ ]:
## Data Distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the number of missing values in each column
missing_values = df_train_cdna_data_cleaned.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (missing_values / len(df_train_cdna_data_cleaned)) * 100

# Create a DataFrame to store the results
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

# Sort the columns by the percentage of missing values
missing_data = missing_data.sort_values(by='Percentage', ascending=False)

# Display the first few rows of the missing data
print(missing_data.head())

# Plotting the missing data as a bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_data.index, y='Percentage', data=missing_data)
plt.xticks(rotation=90)
plt.title('Percentage of Missing Values in Each Column')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.show()

## Mean Imputation & Normalizing Features

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Select only numerical columns
numerical_columns = df_train_cdna_data_cleaned.select_dtypes(include=['number']).columns

# Convert numerical columns to a NumPy array for faster processing
numerical_data = df_train_cdna_data_cleaned[numerical_columns].to_numpy()

# Step 1: Impute missing values with column mean
column_means = np.nanmean(numerical_data, axis=0)  # Compute mean for each column
imputed_data = np.where(np.isnan(numerical_data), column_means, numerical_data)  # Replace NaNs with column means

In [ ]:
# Step 2: Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(imputed_data)

# Convert back to DataFrame for easier interpretation
df_numerical_normalized = pd.DataFrame(normalized_data, columns=numerical_columns, index=df_train_cdna_data_cleaned.index)

# Step 3: Replace original numerical columns with normalized data in the original DataFrame
df_train_cdna_data_cleaned_final = df_train_cdna_data_cleaned.copy()
df_train_cdna_data_cleaned_final[numerical_columns] = df_numerical_normalized

# Display the shapes and verify
print(f"Original Data Shape: {df_train_cdna_data_cleaned.shape}")
print(f"Final Data Shape: {df_train_cdna_data_cleaned_final.shape}")

In [ ]:
df_train_cdna_data_cleaned_final.head()

### Finding Missing Values from Action History Dataset

In [ ]:
## Data Distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the number of missing values in each column
missing_values = df_train_action_history.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (missing_values / len(df_train_action_history)) * 100

# Create a DataFrame to store the results
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

# Sort the columns by the percentage of missing values
missing_data = missing_data.sort_values(by='Percentage', ascending=False)

# Display the first few rows of the missing data
print(missing_data.head())

# Plotting the missing data as a bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_data.index, y='Percentage', data=missing_data)
plt.xticks(rotation=90)
plt.title('Percentage of Missing Values in Each Column')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.show()

### NOTE: Only 30% (Out of 87 Lakh) emails is opened

### Replacing Null Values from open_timestamp to 0 (Means Not Opened)

In [ ]:
# Fill NaN values in the 'open_timestamp' column with 0
df_train_action_history['open_timestamp'] = df_train_action_history['open_timestamp'].fillna(0)

# Verify the result
print(df_train_action_history['open_timestamp'].isnull().sum())  # Should print 0

### Adding New Column called is_opened (value will be 0 or 1)

In [ ]:
df_train_action_history['is_opened'] = (df_train_action_history['open_timestamp'] != 0).astype(int)

In [ ]:
df_train_action_history.head()

In [ ]:
import pandas as pd

# Make a copy of the original DataFrame to avoid modifying it
df_train_action_history_copy = df_train_action_history.copy()

# Replace 0 in 'open_timestamp' with NaT (Not a Time) before conversion
df_train_action_history_copy['open_timestamp'] = df_train_action_history_copy['open_timestamp'].replace(0, pd.NaT)

# Convert send_timestamp and open_timestamp to datetime
df_train_action_history_copy['send_timestamp'] = pd.to_datetime(df_train_action_history_copy['send_timestamp'], errors='coerce')
df_train_action_history_copy['open_timestamp'] = pd.to_datetime(df_train_action_history_copy['open_timestamp'], errors='coerce')

# Ensure timestamps are localized correctly
def localize_to_ist(column):
    if column.dt.tz is None:  # If the column is not timezone-aware
        return column.dt.tz_localize('UTC').dt.tz_localize(None)
    else:  # If the column is already timezone-aware
        return column.dt.tz_localize(None)

df_train_action_history_copy['send_timestamp'] = localize_to_ist(df_train_action_history_copy['send_timestamp'])
df_train_action_history_copy['open_timestamp'] = localize_to_ist(df_train_action_history_copy['open_timestamp'])

# Define the time slot function to extract the slot ID
def get_time_slot(timestamp):
    if pd.isnull(timestamp):
        return None  # Handle missing/invalid timestamps

    day_of_week = timestamp.weekday()  # Monday=0, Sunday=6
    hour = timestamp.hour
    minute = timestamp.minute
    total_minutes = hour * 60 + minute  # Total minutes since midnight

    # Check if time is within 9:00 AM (540 mins) to 9:00 PM (1260 mins)
    if total_minutes < 540 or total_minutes >= 1260:
        return None  # Outside the defined slots

    # Calculate minutes since 9:00 AM and determine the slot
    time_since_9am = total_minutes - 540
    slot = time_since_9am // 180  # Integer division by 180 mins (3 hours)

    # Compute slot ID (1-28)
    slot_id = day_of_week * 4 + slot + 1
    slot_id = f"slot_{slot_id}"
    return slot_id

# Extract features: Day of the week, Hour of the day, and Time Slot ID
df_train_action_history_copy['send_day_of_week'] = df_train_action_history_copy['send_timestamp'].dt.dayofweek
df_train_action_history_copy['send_hour_of_day'] = df_train_action_history_copy['send_timestamp'].dt.hour
df_train_action_history_copy['send_slot'] = df_train_action_history_copy['send_timestamp'].apply(get_time_slot)

# Map open_slot similarly, handling NaT or missing values
df_train_action_history_copy['open_slot'] = df_train_action_history_copy['open_timestamp'].apply(
    lambda x: get_time_slot(x) if pd.notnull(x) else None
)

# Display the new DataFrame with the extracted features
print(df_train_action_history_copy[['send_timestamp', 'send_slot', 'open_timestamp', 'open_slot']].head())


In [ ]:
# Replace NaT and NaN values with 0 for both open_timestamp and open_slot
df_train_action_history_copy['open_timestamp'] = df_train_action_history_copy['open_timestamp'].fillna(0)
df_train_action_history_copy['open_slot'] = df_train_action_history_copy['open_slot'].fillna('no_open')

# Display the new DataFrame with the extracted features
print(df_train_action_history_copy.head())

### NOTE: send_day_of_week value (Monday=0 to Sunday=6)

- slot_1 represents the first time slot (Monday 9:00 AM- 12:00 PM)
- slot_2 represents the second time slot (Monday 12:00 PM – 3:00 PM)
.
.
.
- slot_28 represents the 28th time slot (Sunday 6:00 PM – 9:00 PM)
- NOTE: no_open Means Not opened

In [ ]:
df_train_action_history_copy = df_train_action_history_copy.dropna()

In [ ]:
df_train_action_history_copy.shape

In [ ]:
import numpy as np
unique_vals_of_send_slot = np.unique(df_train_action_history_copy['send_slot'])
print(unique_vals_of_send_slot)
print(len(unique_vals_of_send_slot))

## Finding Unique Customer codes in Action History Dataset

In [ ]:
unique_customer_codes = df_train_action_history_copy['customer_code'].unique()

# To check the number of unique customer codes
print(f"Number of unique customer codes: {len(unique_customer_codes)}")

## Finding Unique Customer codes in CDNA Dataset

In [ ]:
unique_customer_codes = df_train_cdna_data_cleaned_final['CUSTOMER_CODE'].unique()

# To check the number of unique customer codes
print(f"Number of unique customer codes: {len(unique_customer_codes)}")

In [ ]:
print(df_train_cdna_data_cleaned_final.shape)

In [ ]:
print(df_train_action_history_copy.shape)

### Inner Join on Action History Dataset & CDNA dataset

In [ ]:
# Merge the two datasets based on the matching customer_code using an inner join
merged_df = pd.merge(
    df_train_action_history_copy,
    df_train_cdna_data_cleaned_final,
    left_on='customer_code',
    right_on='CUSTOMER_CODE',
    how='inner'  # 'inner' will keep only the rows with matching customer_code
)

# Drop rows with any missing values (NaN) in the merged DataFrame
merged_df_clean = merged_df.dropna()

# Print the shape of the resulting DataFrame
print(f"Merged DataFrame shape after dropping missing values: {merged_df_clean.shape}")

# Display the first few rows of the merged DataFrame
print(merged_df_clean.head())

## 27 Lakh Rows AND 113 Columns

In [ ]:
merged_df_clean.shape

In [ ]:
merged_df_clean.columns

In [ ]:
import pandas as pd

# Group by 'customer_code'
customer_stats = merged_df_clean.groupby('customer_code').agg(
    total_emails_sent=pd.NamedAgg(column='customer_code', aggfunc='size'),  # Count rows for total emails sent
    total_emails_opened=pd.NamedAgg(column='is_opened', aggfunc='sum'),    # Sum 'is_opened' for total opened emails
    open_rate=pd.NamedAgg(column='is_opened', aggfunc=lambda x: (x.sum() / x.size) * 100),  # Calculate open rate (%)
    preferred_time_slot=pd.NamedAgg(column='open_slot', aggfunc=lambda x: x.mode()[0] if not x.mode().empty else None),  # Most common open slot
    most_interacted_category=pd.NamedAgg(column='product_category', aggfunc=lambda x: x.mode()[0] if not x.mode().empty else None),  # Most common category
    most_interacted_subcategory=pd.NamedAgg(column='product_sub_category', aggfunc=lambda x: x.mode()[0] if not x.mode().empty else None)  # Most common subcategory
).reset_index()

# Print the first few rows of the resulting DataFrame
print(customer_stats.head())


In [ ]:
# Merge the customer-level metrics back into the original DataFrame
final_train_dataset = pd.merge(
    merged_df_clean,
    customer_stats,
    on='customer_code',
    how='left'  # Keep all rows in merged_df_clean
)

# Print the shape and preview of the final DataFrame
print(f"Final dataset shape: {final_train_dataset.shape}")
print(final_train_dataset.head())

In [ ]:
final_train_dataset.shape

In [ ]:
final_train_dataset.head()

In [ ]:
list(final_train_dataset.columns)

In [ ]:
columns_to_normalize = ['batch_id','send_day_of_week','send_hour_of_day','total_emails_sent', 'total_emails_opened', 'open_rate']

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Normalize specified columns
normalized_values = scaler.fit_transform(final_train_dataset[columns_to_normalize])

# Create a new DataFrame with normalized columns
normalized_df = final_train_dataset.copy()
normalized_df[columns_to_normalize] = normalized_values

# Display the new DataFrame with normalized values
normalized_df.head()

In [ ]:
normalized_df['customer_code'][0]

In [ ]:
normalized_df['CUSTOMER_CODE'][0]

In [ ]:
del normalized_df['CUSTOMER_CODE']

In [ ]:
list(normalized_df.columns)

In [ ]:
len(normalized_df.columns)

In [ ]:
df_train_final = normalized_df

In [ ]:
df_train_final.shape

In [ ]:
df_train_final = df_train_final.dropna()

In [ ]:
df_train_final.shape

In [ ]:
columns_to_drop = ['send_timestamp', 'open_timestamp']
df_train_final = df_train_final.drop(columns=columns_to_drop, errors='ignore')  # Drop specified columns

In [ ]:
df_train_final.shape

In [ ]:
import numpy as np
np.unique(df_train_final['send_slot'])

In [ ]:
len(np.unique(df_train_final['send_slot']))

In [ ]:
df_train_final.shape

In [ ]:
list(df_train_final.columns)

In [ ]:
Y = df_train_final['send_slot']
X = df_train_final.drop(columns=['send_slot'])

# Display the shapes of X and Y to verify
print(f"Features (X) shape: {X.shape}")
print(f"Target (Y) shape: {Y.shape}")

In [ ]:
def identify_columns(dataframe):
    categorical_columns = dataframe.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_columns = dataframe.select_dtypes(include=['number']).columns.tolist()

    return categorical_columns, numerical_columns

# Identify columns
categorical_columns, numerical_columns = identify_columns(X)

In [ ]:
print("categorical_columns: ", categorical_columns)
print("numerical_columns: ", numerical_columns)

In [ ]:
# Function to check if a column is normalized
def check_normalization(df, column):
    values = df[column]
    if values.min() < 0 or values.max() > 1:  # Check range 0 to 1
        return False
    return True

# Validate normalization for all numerical columns
non_normalized_columns = [
    col for col in numerical_columns if not check_normalization(df_train_final, col)
]

# Print columns that are not normalized
if non_normalized_columns:
    print("Columns not normalized: ", non_normalized_columns)
else:
    print("All numerical columns are normalized!")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler instance
scaler = MinMaxScaler()

# Normalize the 'v288' column
X['v288'] = scaler.fit_transform(X[['v288']])

# Confirm the normalization
print(f"v288 Min: {X['v288'].min()}, Max: {X['v288'].max()}")

In [ ]:
# Function to check if a column is normalized
def check_normalization(df, column):
    values = df[column]
    if values.min() < 0 or values.max() > 1:  # Check range 0 to 1
        return False
    return True

# Validate normalization for all numerical columns
non_normalized_columns = [
    col for col in numerical_columns if not check_normalization(X, col)
]

# Print columns that are not normalized
if non_normalized_columns:
    print("Columns not normalized: ", non_normalized_columns)
else:
    print("All numerical columns are normalized!")

In [ ]:
print("categorical_columns: ", categorical_columns)

In [ ]:
print("Categorical columns and their unique data types:")
for col in categorical_columns:
    print(f"{col}: {X[col].dtype}")

In [ ]:
X[['customer_code','Offer_id','Offer_subid','product_category','product_sub_category',
'open_slot',
'v2',
'v5',
'v6',
'v7',
'v9',
'v10',
'v11',
'v27',
'v29']].head()

In [ ]:
np.unique(X['open_slot'])

In [ ]:
X = X.drop(columns=['open_slot'])

In [ ]:
X.shape

In [ ]:
# Identify columns with mixed data types
mixed_type_columns = []
categorical_columns, numerical_columns = identify_columns(X)
for col in categorical_columns:
    unique_types = X[col].map(type).nunique()  # Count the number of unique data types
    if unique_types > 1:
        mixed_type_columns.append(col)

print("Columns with mixed data types:", mixed_type_columns)

In [ ]:
X[['v30', 'v229']].head()

In [ ]:
X = X.drop(columns=['v30', 'v229'])

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns in X
categorical_columns = X.select_dtypes(include=['object', 'category']).columns

# Create a LabelEncoder instance
label_encoders = {}

# Apply Label Encoding to all categorical columns
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()  # Save the encoder for future use if needed
    X[col] = label_encoders[col].fit_transform(X[col])

In [ ]:
X.head()

In [ ]:
label_encoders

In [ ]:
categorical_columns, numerical_columns = identify_columns(X)
print("categorical_columns: ", categorical_columns)
print("numerical_columns: ", numerical_columns)

In [ ]:
np.unique(Y)

In [ ]:
Y.head()

In [ ]:
# Encode Y using LabelEncoder
y_label_encoder = LabelEncoder()  # Encoder for Y
Y_encoded = y_label_encoder.fit_transform(Y)

# Save the encoder for future use
label_encoders['Y'] = y_label_encoder

# Optionally, print out the mapping for Y (for reference)
y_mapping = dict(zip(y_label_encoder.classes_, y_label_encoder.transform(y_label_encoder.classes_)))
print("Label mapping for Y:", y_mapping)


In [ ]:
## Saving Label Encoder
import joblib
import pickle

# Save LabelEncoders using joblib
joblib.dump(label_encoders, 'label_encoders.pkl')

In [ ]:
X.shape

In [ ]:
Y_encoded.shape

In [ ]:
# Exporting Final Train Data set to CSV
import pandas as pd

# Step 1: Convert Y_encoded to a DataFrame (if it's not already a DataFrame)
# Assuming Y_encoded is a 1D array of encoded labels (e.g., from LabelEncoder)
Y_encoded_df = pd.DataFrame(Y_encoded, columns=['send_slot'])

# Step 2: Merge X and Y_encoded
X_combined = pd.concat([X, Y_encoded_df], axis=1)

# Step 3: Save the combined DataFrame to CSV without the index
X_combined.to_csv('Final_TrainDataset.csv', index=False)

In [ ]:
!pip install magic-wormhole

In [ ]:
!wormhole send /kaggle/working/Final_TrainDataset.csv

In [ ]:
!wormhole send /kaggle/working/label_encoders.pkl

In [ ]:
!pip install magic-wormhole

In [ ]:
!wormhole receive 5-coherence-unearth --accept-file

In [ ]:
!wormhole receive 5-maritime-baboon --accept-file

In [ ]:
import pandas as pd
df_final_train = pd.read_csv('/kaggle/input/team-winners-dataset/Final_TrainDataset.csv')

In [ ]:
df_final_train.shape

In [ ]:
df_final_train.head()

In [ ]:
df_final_train['batch_date'].head()

In [ ]:
df_final_train['batch_id'].head()

In [ ]:
del df_final_train['customer_code']
del df_final_train['batch_date']
del df_final_train['batch_id']

In [ ]:
label_encoders

In [ ]:
del df_final_train['preferred_time_slot']

In [ ]:
df_final_train.head()

In [ ]:
Y_encoded = df_final_train['send_slot']
X = df_final_train.drop(columns=['send_slot'])

# Display the shapes of X and Y to verify
print(f"Features (X) shape: {X.shape}")
print(f"Target (Y) shape: {Y_encoded.shape}")

In [ ]:
# Count the total number of rows
total_rows = X.shape[0]

# Count the number of rows where is_opened is 1 and 0
is_opened_counts = X['is_opened'].value_counts()

# Calculate percentages for each value (1 and 0)
is_opened_percentages = (is_opened_counts / total_rows) * 100

# Display the percentages
print("Percentage of rows where is_opened is 1:", is_opened_percentages.get(1, 0), "%")
print("Percentage of rows where is_opened is 0:", is_opened_percentages.get(0, 0), "%")

In [ ]:
cols_with_missing_vals=['v5', 'v60', 'v7', 'v286', 'v283', 'v279', 'v278', 'v284', 'v285', 'v273', 'v272', 'v275', 'v271', 'v282', 'v281', 'v280', 'v276', 'v277', 'v274', 'v230', 'v35', 'v33', 'v34', 'v102', 'v10', 'v29', 'v73', 'v43', 'v27', 'v31', 'v54', 'v2', 'v101', 'v56', 'v9', 'v37', 'v6', 'v81', 'v11']

In [ ]:
X = X.drop(columns=cols_with_missing_vals)

In [ ]:
X.shape

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_encoded, test_size=0.2, random_state=42)

# Step 2: Convert Y labels to one-hot encoding
Y_train_one_hot = to_categorical(Y_train)
Y_test_one_hot = to_categorical(Y_test)

# Step 3: Calculate class weights for handling imbalance
class_counts = np.bincount(Y_encoded)
total_samples = sum(class_counts)
class_weights = {i: (1 / (count / total_samples)) for i, count in enumerate(class_counts)}
class_weights = {k: v / min(class_weights.values()) for k, v in class_weights.items()}  # Normalize weights

# Step 4: Define the Enhanced Neural Network Model
from tensorflow.keras.regularizers import l2

def create_enhanced_model(input_dim, num_classes=28):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),

        # Feature normalization
        tf.keras.layers.BatchNormalization(),

        # Block 1
        tf.keras.layers.Dense(512, activation='swish', kernel_regularizer=l2(1e-4)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),

        # Block 2
        tf.keras.layers.Dense(384, activation='swish', kernel_regularizer=l2(1e-4)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),

        # Block 3
        tf.keras.layers.Dense(256, activation='swish', kernel_regularizer=l2(1e-4)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),

        # Output
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    # Focal Loss implementation
    def focal_loss(gamma=2.0, alpha=0.25, label_smoothing=0.1):
        def loss_fn(y_true, y_pred):
            epsilon = tf.keras.backend.epsilon()
            y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
            y_true = y_true * (1 - label_smoothing) + label_smoothing / num_classes
            cross_entropy = -y_true * tf.math.log(y_pred)
            loss = alpha * tf.pow(1 - y_pred, gamma) * cross_entropy
            return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))
        return loss_fn

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-4),
        loss=focal_loss(),
        metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
    )
    return model

# Step 5: Create the model with the correct input dimension
input_dim = X_train.shape[1]  # The number of features in X
num_classes = 28

model = create_enhanced_model(input_dim, num_classes)

# Step 6: Training Configuration
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_top3_acc',
        patience=15,
        mode='max',
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_top3_acc',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        mode='max'
    ),
    tf.keras.callbacks.TerminateOnNaN()
]

# Step 7: Train the model
history = model.fit(
    X_train, Y_train_one_hot,
    epochs=200,
    batch_size=2048,  # Increased batch size for better GPU utilization
    validation_data=(X_test, Y_test_one_hot),
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# Step 8: Evaluate the model on the test data
test_loss, test_acc, test_top3_acc = model.evaluate(X_test, Y_test_one_hot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)
print("Test Top-3 Accuracy:", test_top3_acc)

# Step 9: Plot the training history
plt.figure(figsize=(12, 5))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.legend()

plt.show()

In [ ]:
model.save('FinalModel.keras')

In [ ]:
!wormhole send /kaggle/working/FinalModel.keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import register_keras_serializable

# Step 1: Define the custom loss function with serialization support
@register_keras_serializable(package="CustomLoss")
def focal_loss(gamma=2.0, alpha=0.25, label_smoothing=0.1):
    def loss_fn(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        y_true = y_true * (1 - label_smoothing) + label_smoothing / num_classes
        cross_entropy = -y_true * tf.math.log(y_pred)
        loss = alpha * tf.pow(1 - y_pred, gamma) * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))
    return loss_fn

# Step 2: Define the number of classes (must match the original model)
num_classes = 28  # Replace with the actual number of classes used during training

# Step 3: Load the saved model
loaded_model = tf.keras.models.load_model('FinalModel.keras', compile=False)

# Step 2: Recompile the model with a standard loss function (not used during inference)
loaded_model.compile(
    optimizer='adam',  # Any optimizer can be used since it won't be used for inference
    loss='categorical_crossentropy',  # Standard loss function (not used during inference)
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
)

# Step 4: Evaluate the loaded model
test_loss, test_acc, test_top3_acc = loaded_model.evaluate(X_test, Y_test_one_hot)
print("Test Loss (Loaded Model):", test_loss)
print("Test Accuracy (Loaded Model):", test_acc)
print("Test Top-3 Accuracy (Loaded Model):", test_top3_acc)

In [ ]:
list(X.columns)

In [ ]:
len(X.columns)

In [ ]:
# Export X.columns to a CSV file
X.columns.to_series().to_csv('relevant_columns_for_finalmodel.csv', index=False, header=False)

print("Column names exported to 'relevant_columns_for_finalmodel.csv'")

In [ ]:
# Load the column names from the exported file
with open('relevant_columns_for_finalmodel.csv', 'r') as file:
    train_columns = file.read().splitlines()

print("Loaded column names:", train_columns)
print("Loaded column len", len(train_columns))

In [ ]:
!wormhole send /kaggle/working/relevant_columns_for_finalmodel.csv

In [ ]:
import os
os.listdir('/kaggle/working/')

In [ ]:
os.remove('/kaggle/working/FinalModel.keras')

In [ ]:
import pandas as pd
df_train_cdna_data = pd.read_csv('/kaggle/input/team-winners-dataset/train_cdna_data.csv')
# Step 1: Identify numerical columns
numerical_columns = df_train_cdna_data.select_dtypes(include=['number'])
mean_values = numerical_columns.mean()

# Step 2: Save the means in a single row in a CSV file
mean_values.T.to_frame().T.to_csv('numerical_column_means.csv', index=False)

print("Numerical column means saved to 'numerical_column_means.csv' as a single row.")

In [ ]:
mean_values_row = pd.read_csv('numerical_column_means.csv')
mean_values_row.head()

In [ ]:
!pip install magic-wormhole

In [ ]:
!wormhole send /kaggle/working/numerical_column_means.csv

In [ ]:
!wormhole receive 3-phonetic-rocker --accept-file

In [ ]:
import tensorflow as tf
# Step 3: Load the saved model
loaded_model = tf.keras.models.load_model('FinalModel.keras', compile=False)

# Step 2: Recompile the model with a standard loss function (not used during inference)
loaded_model.compile(
    optimizer='adam',  # Any optimizer can be used since it won't be used for inference
    loss='categorical_crossentropy',  # Standard loss function (not used during inference)
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
)

In [ ]:
import pandas as pd
df_test_action_history = pd.read_csv('/kaggle/input/team-winners-dataset/test_action_history.csv')
df_test_cdna_data = pd.read_csv('/kaggle/input/team-winners-dataset/test_cdna_data.csv')
df_test_customers = pd.read_csv('/kaggle/input/team-winners-dataset/test_customers.csv')


In [ ]:
df_test_action_history.head()

In [ ]:
df_test_action_history.shape

In [ ]:
df_test_cdna_data.shape

In [ ]:
df_test_cdna_data.head()

In [ ]:
# Fill NaN values in the 'open_timestamp' column with 0
df_test_action_history['open_timestamp'] = df_test_action_history['open_timestamp'].fillna(0)
df_test_action_history['is_opened'] = (df_test_action_history['open_timestamp'] != 0).astype(int)

In [ ]:
df_test_action_history.head()

In [ ]:
# Count the number of null values per row
df_test_cdna_data['null_count'] = df_test_cdna_data.isnull().sum(axis=1)

# Sort by CUSTOMER_CODE and then by null_count (ascending) to prioritize rows with the least null values
df_test_cdna_data_sorted = df_test_cdna_data.sort_values(by=['CUSTOMER_CODE', 'null_count'])

# Drop duplicates, keeping the first (least null) row for each CUSTOMER_CODE
df_test_cdna_data_unique = df_test_cdna_data_sorted.drop_duplicates(subset='CUSTOMER_CODE', keep='first')

# Drop the 'null_count' column as it's no longer needed
df_test_cdna_data_unique = df_test_cdna_data_unique.drop(columns=['null_count'])

# Reset the index
df_test_cdna_data_unique.reset_index(drop=True, inplace=True)

# Display the shape of the resulting DataFrame and a sample
print(df_test_cdna_data_unique.shape)
print(df_test_cdna_data_unique.head())

In [ ]:
df_test_customers.head()

In [ ]:
# Find common customer_code values in both DataFrames
common_customer_codes = set(df_test_cdna_data_unique['CUSTOMER_CODE']).intersection(df_test_customers['CUSTOMER_CODE'])

# Count the occurrences of these customer codes in both DataFrames
count_cdna_data = df_test_cdna_data_unique[df_test_cdna_data_unique['CUSTOMER_CODE'].isin(common_customer_codes)].shape[0]
count_customers = df_test_customers[df_test_customers['CUSTOMER_CODE'].isin(common_customer_codes)].shape[0]

# Display the results
print(f"Count of common customer_code in df_test_cdna_data_unique: {count_cdna_data}")
print(f"Count of common customer_code in df_test_customers: {count_customers}")


In [ ]:
df_train_cdna = pd.read_csv('/kaggle/input/team-winners-dataset/train_cdna_data.csv')

In [ ]:

# Find common customer_code values in both DataFrames
common_customer_codes = set(df_train_cdna['CUSTOMER_CODE']).intersection(df_test_customers['CUSTOMER_CODE'])

# Count the occurrences of these customer codes in both DataFrames
count_final_train_data = df_train_cdna[df_train_cdna['CUSTOMER_CODE'].isin(common_customer_codes)].shape[0]
count_customers = df_test_customers[df_test_customers['CUSTOMER_CODE'].isin(common_customer_codes)].shape[0]

# Display the results
print(f"Count of common customer_code in df_test_cdna_data_unique: {count_final_train_data}")
print(f"Count of common customer_code in df_test_customers: {count_customers}")

## NOTE: Since train customer code and test customer code is different, so customer_code can't be a feature

In [ ]:
import pandas as pd


# Replace 0 in 'open_timestamp' with NaT (Not a Time) before conversion
df_test_action_history['open_timestamp'] = df_test_action_history['open_timestamp'].replace(0, pd.NaT)

# Convert send_timestamp and open_timestamp to datetime
df_test_action_history['send_timestamp'] = pd.to_datetime(df_test_action_history['send_timestamp'], errors='coerce')
df_test_action_history['open_timestamp'] = pd.to_datetime(df_test_action_history['open_timestamp'], errors='coerce')

# Ensure timestamps are localized correctly
def localize_to_ist(column):
    if column.dt.tz is None:  # If the column is not timezone-aware
        return column.dt.tz_localize('UTC').dt.tz_localize(None)
    else:  # If the column is already timezone-aware
        return column.dt.tz_localize(None)

df_test_action_history['send_timestamp'] = localize_to_ist(df_test_action_history['send_timestamp'])
df_test_action_history['open_timestamp'] = localize_to_ist(df_test_action_history['open_timestamp'])

# Define the time slot function to extract the slot ID
def get_time_slot(timestamp):
    if pd.isnull(timestamp):
        return None  # Handle missing/invalid timestamps

    day_of_week = timestamp.weekday()  # Monday=0, Sunday=6
    hour = timestamp.hour
    minute = timestamp.minute
    total_minutes = hour * 60 + minute  # Total minutes since midnight

    # Check if time is within 9:00 AM (540 mins) to 9:00 PM (1260 mins)
    if total_minutes < 540 or total_minutes >= 1260:
        return None  # Outside the defined slots

    # Calculate minutes since 9:00 AM and determine the slot
    time_since_9am = total_minutes - 540
    slot = time_since_9am // 180  # Integer division by 180 mins (3 hours)

    # Compute slot ID (1-28)
    slot_id = day_of_week * 4 + slot + 1
    slot_id = f"slot_{slot_id}"
    return slot_id

# Extract features: Day of the week, Hour of the day, and Time Slot ID
df_test_action_history['send_day_of_week'] = df_test_action_history['send_timestamp'].dt.dayofweek
df_test_action_history['send_hour_of_day'] = df_test_action_history['send_timestamp'].dt.hour
df_test_action_history['send_slot'] = df_test_action_history['send_timestamp'].apply(get_time_slot)

# Map open_slot similarly, handling NaT or missing values
df_test_action_history['open_slot'] = df_test_action_history['open_timestamp'].apply(
    lambda x: get_time_slot(x) if pd.notnull(x) else None
)

# Display the new DataFrame with the extracted features
print(df_test_action_history[['send_timestamp', 'send_slot', 'open_timestamp', 'open_slot']].head())


In [ ]:
# Replace NaT and NaN values with 0 for both open_timestamp and open_slot
df_test_action_history['open_timestamp'] = df_test_action_history['open_timestamp'].fillna(0)
df_test_action_history['open_slot'] = df_test_action_history['open_slot'].fillna('no_open')

# Display the new DataFrame with the extracted features
print(df_test_action_history.head())

In [ ]:
df_test_action_history.shape

In [ ]:
df_test_action_history = df_test_action_history.dropna()
df_test_action_history.shape

In [ ]:
unique_customer_codes = df_test_action_history['customer_code'].unique()

# To check the number of unique customer codes
print(f"Number of unique customer codes: {len(unique_customer_codes)}")

In [ ]:
unique_customer_codes = df_test_cdna_data_unique['CUSTOMER_CODE'].unique()

# To check the number of unique customer codes
print(f"Number of unique customer codes: {len(unique_customer_codes)}")

In [ ]:
df_test_action_history['customer_code'][0]

In [ ]:
df_test_action_history.shape

In [ ]:
df_test_cdna_data_unique.shape

In [ ]:
len(X.columns)

In [ ]:
relevant_cols = list(X.columns) + ['send_slot']

In [ ]:
print(relevant_cols)

In [ ]:
print("Unique customer_code in df_test_action_history:")
print(len(df_test_action_history['customer_code'].unique()))

print("Unique CUSTOMER_CODE in df_test_cdna_data_unique:")
print(len(df_test_cdna_data_unique['CUSTOMER_CODE'].unique()))

# Check intersection of both columns
overlap = set(df_test_action_history['customer_code']).intersection(set(df_test_cdna_data_unique['CUSTOMER_CODE']))
print(f"Overlapping customer_code values code: {len(overlap)}")


In [ ]:
# Add 'CUSTOMER_CODE' to relevant_cols if it's not already present
if 'CUSTOMER_CODE' not in relevant_cols:
    relevant_cols.append('CUSTOMER_CODE')

# Create a list to store valid columns
valid_columns = []

# Loop through relevant_cols to check if each column exists in df_test_cdna_data_unique
for col in relevant_cols:
    if col in df_test_cdna_data_unique.columns:
        valid_columns.append(col)  # Keep only columns that exist

# Filter the DataFrame to retain only the valid columns
df_test_cdna_data_unique = df_test_cdna_data_unique[valid_columns]

In [ ]:
df_test_cdna_data_unique.shape

In [ ]:
df_test_cdna_data_unique.shape

In [ ]:
df_test_cdna_data_unique.head()

In [ ]:
df_test_action_history.shape

In [ ]:
df_test_action_history = df_test_action_history.dropna()

In [ ]:
df_test_action_history.shape

In [ ]:
## Data Distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the number of missing values in each column
missing_values = df_test_action_history.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (missing_values / len(df_test_action_history)) * 100

# Create a DataFrame to store the results
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

# Sort the columns by the percentage of missing values
missing_data = missing_data.sort_values(by='Percentage', ascending=False)

# Display the first few rows of the missing data
print(missing_data.head())

# Plotting the missing data as a bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_data.index, y='Percentage', data=missing_data)
plt.xticks(rotation=90)
plt.title('Percentage of Missing Values in Each Column')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.show()

In [ ]:
len(X.columns)

In [ ]:
# Get the common columns between X.columns and df_test_cdna_data_unique columns
common_columns = df_test_cdna_data_unique.columns.intersection(X.columns)

# Subset df_test_cdna_data_unique to keep only the common columns
df_test_cdna_data_unique = df_test_cdna_data_unique[common_columns]

print("Dropped irrelevant columns successfully!")

In [ ]:
## Data Distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the number of missing values in each column
missing_values = df_test_cdna_data_unique.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (missing_values / len(df_test_cdna_data_unique)) * 100

# Create a DataFrame to store the results
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

# Sort the columns by the percentage of missing values
missing_data = missing_data.sort_values(by='Percentage', ascending=False)

# Display the first few rows of the missing data
print(missing_data.head())

# Plotting the missing data as a bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_data.index, y='Percentage', data=missing_data)
plt.xticks(rotation=90)
plt.title('Percentage of Missing Values in Each Column')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.show()

In [ ]:

df_test_cdna_data_unique.shape

In [ ]:
df_train_cdna_data = pd.read_csv('/kaggle/input/team-winners-dataset/train_cdna_data.csv')

In [ ]:
df_train_cdna_data.head()

In [ ]:
common_columns = df_test_cdna_data_unique.columns.intersection(df_train_cdna_data.columns)
print(len(common_columns))

In [ ]:
df_train_cdna_data.head()

In [ ]:
# Ensure the columns of interest overlap between the two DataFrames
common_columns = df_test_cdna_data_unique.columns.intersection(df_train_cdna_data.columns)

numerical_cols_count = 0
# Loop through the common columns and fill NaN in df_test_cdna_data_unique with the mean of df_train_cdna_data
for col in common_columns:
    if df_train_cdna_data[col].dtype in ['float64', 'int64']:  # Ensure column is numeric
        mean_value = df_train_cdna_data[col].mean()
        df_test_cdna_data_unique[col].fillna(mean_value, inplace=True)
        numerical_cols_count += 1

# Check for any remaining NaN values in the test DataFrame
print(f"Remaining NaN values in df_test_cdna_data_unique: {df_test_cdna_data_unique.isna().sum().sum()}")
print('numerical_cols_count: ', numerical_cols_count)
# Optionally, display the updated DataFrame
print(df_test_cdna_data_unique.head())

In [ ]:
## Data Distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the number of missing values in each column
missing_values = df_test_cdna_data_unique.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (missing_values / len(df_test_cdna_data_unique)) * 100

# Create a DataFrame to store the results
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

# Sort the columns by the percentage of missing values
missing_data = missing_data.sort_values(by='Percentage', ascending=False)

# Display the first few rows of the missing data
print(missing_data.head())

# Plotting the missing data as a bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_data.index, y='Percentage', data=missing_data)
plt.xticks(rotation=90)
plt.title('Percentage of Missing Values in Each Column')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.show()

In [ ]:
df_test_cdna_data_unique.shape

In [ ]:
label_encoders

In [ ]:
import joblib
label_encoders = joblib.load('/kaggle/input/team-winners-dataset/label_encoders.pkl')

In [ ]:
label_encoders

In [ ]:
# Iterate over each column in df_test_cdna_data_unique
for column in df_test_cdna_data_unique.columns:
    # Check if the column is in label_encoders
    if column in label_encoders:
        # Apply the LabelEncoder to the column
        df_test_cdna_data_unique[column] = label_encoders[column].fit_transform(df_test_cdna_data_unique[column])

# Display the first few rows after encoding
print(df_test_cdna_data_unique.head())

In [ ]:
df_test_action_history.head()

In [ ]:
# Iterate over each column in df_test_cdna_data_unique
for column in df_test_action_history.columns:
    # Check if the column is in label_encoders
    if column in label_encoders and column != 'customer_code':
        # Apply the LabelEncoder to the column
        df_test_action_history[column] = label_encoders[column].fit_transform(df_test_action_history[column])

# Display the first few rows after encoding
print(df_test_action_history.head())

In [ ]:
df_test_action_history.head()

In [ ]:
## Data Distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the number of missing values in each column
missing_values = df_test_cdna_data_unique.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (missing_values / len(df_test_cdna_data_unique)) * 100

# Create a DataFrame to store the results
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

# Sort the columns by the percentage of missing values
missing_data = missing_data.sort_values(by='Percentage', ascending=False)

# Display the first few rows of the missing data
print(missing_data.head())

# Plotting the missing data as a bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_data.index, y='Percentage', data=missing_data)
plt.xticks(rotation=90)
plt.title('Percentage of Missing Values in Each Column')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.show()

In [ ]:
df_test_action_history.shape

In [ ]:
df_test_cdna_data_unique.shape

In [ ]:
df_test_action_history.shape

In [ ]:
df_test_cdna_data_unique.shape

In [ ]:
# Merge the two datasets based on the matching customer_code using an inner join
merged_df_test = pd.merge(
    df_test_action_history,
    df_test_cdna_data_unique,
    left_on='customer_code',
    right_on='CUSTOMER_CODE',
    how='inner'  # 'inner' will keep only the rows with matching customer_code
)

# Print the shape of the resulting DataFrame
print(f"Merged DataFrame shape: {merged_df_test.shape}")

# Display the first few rows of the merged DataFrame
print(merged_df_test.head())

In [ ]:
import pandas as pd

# Group by 'customer_code'
customer_stats_test = merged_df_test.groupby('customer_code').agg(
    total_emails_sent=pd.NamedAgg(column='customer_code', aggfunc='size'),  # Count rows for total emails sent
    total_emails_opened=pd.NamedAgg(column='is_opened', aggfunc='sum'),    # Sum 'is_opened' for total opened emails
    open_rate=pd.NamedAgg(column='is_opened', aggfunc=lambda x: (x.sum() / x.size) * 100),  # Calculate open rate (%)
    preferred_time_slot=pd.NamedAgg(column='open_slot', aggfunc=lambda x: x.mode()[0] if not x.mode().empty else None),  # Most common open slot
    most_interacted_category=pd.NamedAgg(column='product_category', aggfunc=lambda x: x.mode()[0] if not x.mode().empty else None),  # Most common category
    most_interacted_subcategory=pd.NamedAgg(column='product_sub_category', aggfunc=lambda x: x.mode()[0] if not x.mode().empty else None)  # Most common subcategory
).reset_index()

# Print the first few rows of the resulting DataFrame
print(customer_stats_test.head())


In [ ]:
# Merge the customer-level metrics back into the original DataFrame
final_test_dataset = pd.merge(
    merged_df_test,
    customer_stats_test,
    on='customer_code',
    how='left'  # Keep all rows in merged_df_clean
)

# Print the shape and preview of the final DataFrame
print(f"Final dataset shape: {final_test_dataset.shape}")
print(final_test_dataset.head())

In [ ]:
'send_slot' in final_test_dataset

In [ ]:
revalent_colus = list(X.columns) + ['send_slot']

In [ ]:
len(revalent_colus)

In [ ]:
customer_codes = final_test_dataset['customer_code']

In [ ]:
customer_codes.shape

In [ ]:
final_test_dataset = final_test_dataset[revalent_colus]

In [ ]:
final_test_dataset.shape

In [ ]:
final_test_dataset['send_slot'].head()

In [ ]:
numerical_columns = final_test_dataset.select_dtypes(include=['number']).columns
print('numerical_columns: ', numerical_columns)
print("len(numerical_columns): ", len(numerical_columns))

In [ ]:

# Exclude columns that are in label_encoders
columns_to_scale = [col for col in numerical_columns if col not in label_encoders]
print("columns_to_scale: ", columns_to_scale)
print("len(columns_to_scale): ", len(columns_to_scale))

In [ ]:
all(columns_to_scale == numerical_columns)

In [ ]:
final_test_dataset[columns_to_scale].head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply the scaler to the selected numerical columns
final_test_dataset[columns_to_scale] = scaler.fit_transform(final_test_dataset[columns_to_scale])

# Check the first few rows of the dataset to confirm the scaling
print(final_test_dataset.head())

In [ ]:
final_test_dataset['send_slot'] = label_encoders['Y'].fit_transform(final_test_dataset['send_slot'])

In [ ]:
final_test_dataset['send_slot'].head()

In [ ]:
numerical_columns = final_test_dataset.select_dtypes(include=['number']).columns
print('numerical_columns: ', numerical_columns)
print("len(numerical_columns): ", len(numerical_columns))

In [ ]:
final_test_dataset.head()

In [ ]:
list(final_test_dataset.columns)

In [ ]:
final_test_dataset.head()

In [ ]:
categorical_cols_count = 0
for column in final_test_dataset.columns:
    # Check if the column is in label_encoders
    if column in label_encoders:
        categorical_cols_count += 1

print("categorical_cols_count: ", categorical_cols_count)

In [ ]:
# Find categorical columns in final_test_dataset
categorical_columns = final_test_dataset.select_dtypes(include=['object', 'category']).columns.tolist()

# Display the categorical columns
print("Categorical columns:", categorical_columns)
print(len(categorical_columns))

In [ ]:
final_test_dataset.shape

In [ ]:
final_test_dataset.head()

In [ ]:
# Iterate over each column in df_test_cdna_data_unique
for column in final_test_dataset.columns:
    # Check if the column is in label_encoders
    if column in label_encoders:
        # Apply the LabelEncoder to the column
        final_test_dataset[column] = label_encoders[column].fit_transform(final_test_dataset[column])

# Display the first few rows after encoding
print(final_test_dataset.head())

In [ ]:
final_test_dataset.head()

In [ ]:
X = final_test_dataset.drop('send_slot', axis=1)  # Dropping the target column
Y = final_test_dataset['send_slot']

In [ ]:
print('X.shape: ', X.shape)
print('Y.shape: ', Y.shape)

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
from tensorflow.keras.utils import to_categorical

Y_encoded = to_categorical(Y)

In [ ]:
test_loss, test_acc, test_top3_acc = loaded_model.evaluate(X, Y_encoded)
print("Test Loss (Loaded Model):", test_loss)
print("Test Accuracy (Loaded Model):", test_acc)
print("Test Top-3 Accuracy (Loaded Model):", test_top3_acc)

In [ ]:
predictions = loaded_model.predict(X)

In [ ]:
# Step 2: Sort each prediction in descending order to get the best slot first
predicted_slots_order_indices = np.argsort(predictions, axis=1)[:, ::-1]  # Sort and reverse order

# Step 3: Decode predicted class indices to class labels using the LabelEncoder
predicted_slots_order = [label_encoders['Y'].inverse_transform(order) for order in predicted_slots_order_indices]

# Display the first few predicted slot orders
print("Predicted Slot Orders (Top 3 shown for the first test sample):", predicted_slots_order[0][:3])

In [ ]:
import pandas as pd

# Example customer_code list (replace with your actual customer codes)
# customer_codes = ['customer_{}'.format(i) for i in range(len(predicted_slots_order))]

# Step 1: Create a DataFrame with customer_code and predicted_slots_order
df_predictions = pd.DataFrame({
    'customer_code': customer_codes,
    'predicted_slots_order': [str(order) for order in predicted_slots_order]
})

# Step 2: Export DataFrame to CSV
df_predictions.to_csv('predicted_slots_order.csv', index=False)

# Display first few rows to confirm
print(df_predictions.head())


In [ ]:
df_predictions.shape

In [ ]:
df_predictions.head()

In [ ]:
df_test_customers.shape

In [ ]:
df_test_customers.head()

In [ ]:
df_test_customers_new = df_test_customers.copy()

In [ ]:
# Step 1: Create a mapping of customer_code to predicted_slots_order from df_predictions
customer_predicted_slots_map = df_predictions.set_index('customer_code')['predicted_slots_order'].to_dict()

# Step 2: Map the predicted_slots_order to df_test_customers based on customer_code
df_test_customers_new['predicted_slots_order'] = df_test_customers_new['CUSTOMER_CODE'].map(customer_predicted_slots_map)

# Step 3: Drop rows where 'predicted_slots_order' is NaN (if any)
final_submission_df = df_test_customers_new.dropna(subset=['predicted_slots_order'])

final_submission_df = final_submission_df.rename(columns={'CUSTOMER_CODE': 'customer_code'})


# Display the first few rows of the final submission DataFrame
print(final_submission_df.head())

# Step 4: Save the final submission to a CSV file
final_submission_df.to_csv('final_submission.csv', index=False)


In [ ]:
final_submission_df.shape